In [30]:
import os
import numpy as np
import pandas as pd
from pandas import DataFrame
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.service import Service as EdgeService

In [31]:
%reload_ext dotenv
%dotenv

In [32]:
CHAVE_NFC                 = os.getenv('CHAVE_NFC')
URL_FAZENDA               = os.getenv('URL_FAZENDA')
TEMPO_ESPERA              = os.getenv('TEMPO_ESPERA')
NOME_CAMPO_CHAVE_ACESSO   = os.getenv('NOME_CAMPO_CHAVE_ACESSO')
NOME_BOTAO_DETALHAR_NOTA  = os.getenv('NOME_BOTAO_DETALHAR_NOTA')
NOME_BOTAO_CONSULTAR_NOTA = os.getenv('NOME_BOTAO_CONSULTAR_NOTA')

In [33]:
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))
driver.maximize_window()
wait = WebDriverWait(driver, TEMPO_ESPERA)

In [34]:
def get_path(
    path: str,
    indice: int = 0,
    should_format_path: bool = True
    ) -> str:
    try:
        path = path.format(i=indice) if should_format_path else path
        return driver.find_element(By.XPATH, path).get_attribute('innerHTML')
    except:
        return ''

In [35]:
def filter_dataframe(
        df: DataFrame,
        coluna: str,
        values_to_ignore
    ) -> DataFrame:
    return df[~df[coluna].isin(values_to_ignore)]

In [36]:
def save_dataframe_to_csv(
        df: DataFrame,
        file_name: str,
        file_type: str,
        nome_coluna_indice: str
    ):
    df.to_csv(f'{file_name}.{file_type}', index_label=nome_coluna_indice, index=True)

In [37]:
def get_info_nota_fiscal(
        url: str,
        chave: str,
        nome_campo_chave: str,
        nome_botao_consulta_nota: str,
        nome_botao_detalhar_nota: str
    ):
    driver.get(url)
    wait.until(EC.element_to_be_clickable((By.NAME, nome_campo_chave))).send_keys(chave)
    wait.until(EC.element_to_be_clickable((By.NAME, nome_botao_consulta_nota))).click()
    wait.until(EC.visibility_of_element_located((By.NAME, nome_botao_detalhar_nota))).click()

In [38]:
def convert_date_to_notion_format(
        df: DataFrame,
        coluna: str,
        file_uses_day_first = True
    ) -> DataFrame:
    df[coluna] = pd.to_datetime(df[coluna], dayfirst=file_uses_day_first).dt.strftime(REGEX_DATA)
    return df

In [39]:
def convert_number_to_notion_format(
        df: DataFrame,
        coluna: str,
    ) -> DataFrame:
    df[coluna] = pd.to_numeric(df[coluna].str.replace(',', '.'))
    return df

In [40]:
should_retry = True

while should_retry:
    try:
        get_info_nota_fiscal(
            url=URL_FAZENDA,
            chave=CHAVE_NFC,
            nome_campo_chave=NOME_CAMPO_CHAVE_ACESSO,
            nome_botao_detalhar_nota=NOME_BOTAO_DETALHAR_NOTA,
            nome_botao_consulta_nota=NOME_BOTAO_CONSULTAR_NOTA
        )
        should_retry = False
    except:
        should_retry = True

In [41]:
TOTAL_ITENS_NOTA         = os.getenv('TOTAL_ITENS_NOTA')
PATH_NUMERO_NOTA         = os.getenv('PATH_NUMERO_NOTA')
PATH_DATA_COMPRA         = os.getenv('PATH_DATA_COMPRA')
PATH_NOME_PRODUTO        = os.getenv('PATH_NOME_PRODUTO')
PATH_QUANTIDADE          = os.getenv('PATH_QUANTIDADE')
PATH_UNIDADE_COMERCIAL   = os.getenv('PATH_UNIDADE_COMERCIAL')
PATH_VALOR_DESCONTO      = os.getenv('PATH_VALOR_DESCONTO')
PATH_VALOR_PRODUTO       = os.getenv('PATH_VALOR_PRODUTO')
PATH_NOME_MERCADO        = os.getenv('PATH_NOME_MERCADO')

NOME_COLUNA_PRODUTO      = os.getenv('NOME_COLUNA_PRODUTO')
NOME_COLUNA_STATUS       = os.getenv('NOME_COLUNA_STATUS')
NOME_COLUNA_COMPRA       = os.getenv('NOME_COLUNA_COMPRA')
NOME_COLUNA_MERCADO      = os.getenv('NOME_COLUNA_MERCADO')
NOME_COLUNA_QUANTIDADE   = os.getenv('NOME_COLUNA_QUANTIDADE')
NOME_COLUNA_VALOR        = os.getenv('NOME_COLUNA_VALOR')
NOME_COLUNA_DESCONTO     = os.getenv('NOME_COLUNA_DESCONTO')
NOME_COLUNA_INDICE       = os.getenv('NOME_COLUNA_INDICE')

VALOR_CAMPO_STATUS       = os.getenv('VALOR_CAMPO_STATUS')
VALOR_CAMPO_SUPERMERCADO = os.getenv('VALOR_CAMPO_SUPERMERCADO')

REGEX_DATA               = os.getenv('REGEX_DATA')
FILE_TYPE_TO_EXPORT      = os.getenv('FILE_TYPE_TO_EXPORT')

In [42]:
list_nota = []

for i in range(len(driver.find_elements(By.XPATH, TOTAL_ITENS_NOTA))):
    list_nota.append([
        get_path(PATH_NOME_PRODUTO, indice=i),
        'Comprado',
        get_path(PATH_DATA_COMPRA, indice=i),
        get_path(PATH_NOME_MERCADO, indice=i),
        get_path(PATH_QUANTIDADE, indice=i),
        get_path(PATH_VALOR_PRODUTO, indice=i),
        get_path(PATH_VALOR_DESCONTO, indice=i+1),
    ])

In [43]:
df = pd.DataFrame(
    list_nota,
    columns=[
        NOME_COLUNA_PRODUTO,
        NOME_COLUNA_STATUS,
        NOME_COLUNA_COMPRA,
        NOME_COLUNA_MERCADO,
        NOME_COLUNA_QUANTIDADE,
        NOME_COLUNA_VALOR,
        NOME_COLUNA_DESCONTO
    ]
)

df = filter_dataframe(
    df,
    NOME_COLUNA_PRODUTO,
    values_to_ignore=['']
)
df = convert_date_to_notion_format(df, NOME_COLUNA_COMPRA)
df = convert_number_to_notion_format(df, NOME_COLUNA_VALOR)
df = convert_number_to_notion_format(df, NOME_COLUNA_DESCONTO)
df = convert_number_to_notion_format(df, NOME_COLUNA_QUANTIDADE)
df.index = np.arange(1, len(df)+1)

if not df.empty:
    save_dataframe_to_csv(
        df,
        get_path(PATH_NUMERO_NOTA, should_format_path=False),
        FILE_TYPE_TO_EXPORT,
        NOME_COLUNA_INDICE
    )